## functools.wrap 을 사용해 함수 데코레이터를 정의하라

파이썬은 함수에 적용할 수 있는 데코레이터(decorator)를 정의하는 특별한 구문을 제공한다.
- 데코레이터는 자신이 감싸고 있는 함수가 호출되기 전과 후의 코드를 추가로 실행해준다.
- 이는 데코레이터가 자신이 감싸고 있는 함수의 입력 인자, 반환 값, 함수에서 발생한 오류에 접근할 수 있다는 뜻이다.
- 함수의 의미를 강화하거나 디버깅을 하거나 함수를 등록하는 등의 일에 이런 기능을 유용하게 쓸 수 있다.

예를 들어 함수가 호출될 때마다 인자 값과 반환 값을 출력하고 싶다고 하자.
- 이런 기능은 재귀적 함수에서 함수 호출이 재귀적으로 내포되는 경우를 디버깅할 때 특히 유용하다.
- 여기서서는 `*args` 와 `**kwargs` 를 사용해 감싸진 함수의 모든 파라미터를 전달받았다.

In [10]:
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) -> {result!r}')
        
        return result
    return wrapper

이 데코레이터를 함수에 적용할 때는 @ 기호를 사용한다.

In [11]:
@trace
def fibonacci(n):
    """n번째 피보나치 수를 반환한다."""
    
    if n in (0, 1):
        return n
    return (fibonacci(n-2) + fibonacci(n-1))

`@` 기호를 사용하는 것은 이 함수에 대해 데코레이터를 호출한 후, 데코레이터가 반환한 결과(데코레이터는 함수를 반환하므로 원래 함수 대신 데코레이터가 반환한 함수를 써도 된다.)를
원래 함수가 속해야 하는 영역에 원래 함수와 같은 이름으로 등록하는 것과 같다.

```python
fibonacci = trace(fibonacci)
```

In [12]:
fibonacci = trace(fibonacci)

이렇게 꾸며진 함수(fibonacci)는 wrapper 코드를 원래의 fibonacci 함수가 실행되기 전과 후에 실행한다.

따라서 wrapper 는 재귀 스택의 매 단계마다 함수의 인자와 반환 값을 출력한다.

In [13]:
fibonacci(4)

fibonacci((0,), {}) -> 0
wrapper((0,), {}) -> 0
fibonacci((1,), {}) -> 1
wrapper((1,), {}) -> 1
fibonacci((2,), {}) -> 1
wrapper((2,), {}) -> 1
fibonacci((1,), {}) -> 1
wrapper((1,), {}) -> 1
fibonacci((0,), {}) -> 0
wrapper((0,), {}) -> 0
fibonacci((1,), {}) -> 1
wrapper((1,), {}) -> 1
fibonacci((2,), {}) -> 1
wrapper((2,), {}) -> 1
fibonacci((3,), {}) -> 2
wrapper((3,), {}) -> 2
fibonacci((4,), {}) -> 3
wrapper((4,), {}) -> 3


3

이 코드는 잘 작동하지만, 의도하지 않은 부작용이 있다. 데코레이터가 반환하는 함수(앞에서 호출한 감싸진 fibonacci 함수)의 이름이 fibonacci 가 아니게 된다.

In [14]:
print(fibonacci)

<function trace.<locals>.wrapper at 0x7ffa41044d30>


이런 결과가 나타난 이유
- trace 함수는 자신의 본문에 정의된 wrapper 함수를 반환한다.
- 데코레이터로 인해 이 wrapper 함수가 모듈에 fibonacci라는 이름으로 등록된다.
- 이런 동작은 디버거와 같이 인트로스펙션(실행 시점에 프로그램이 어떻게 실행되는지 관찰하는 것)을 하는 도구에서 문제가 된다.

예를 들어 꾸며진 fibonacci 함수에 help 내장 함수를 호출하면 쓸모가 없다.

다음과 같이 호출하면 fibonacci 함수의 맨 앞부분에 있는 독스트링이 출력 돼야 하지만, 실제로는 그렇지 않다.

In [15]:
help(fibonacci)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



데코레이터가 감싸고 있는 원래 함수의 위치를 찾을 수 없기 때문에 객체 직렬화도 깨진다.

In [16]:
import pickle

pickle.dumps(fibonacci)

AttributeError: Can't pickle local object 'trace.<locals>.wrapper'

문제를 해결하는 방법은 functools 내장 모듈에 정의된 wraps 도우미 함수를 사용하는 것이다. 이 함수는 데코레이터 작성을 돕는 데코레이터이다.

wraps 를 wrapper 함수에 적용하면 wraps가 데코레이터 내부에 들어가는 함수에서 중요한 메타데이터를 복사해 wrapper 함수에 적용해준다.

In [17]:
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) -> {result!r}')
        
        return result
    return wrapper

@trace
def fibonacci(n):
    """n번째 피보나치 수를 반환한다."""
    
    if n in (0, 1):
        return n
    return (fibonacci(n-2) + fibonacci(n-1))

In [18]:
help(fibonacci)

Help on function fibonacci in module __main__:

fibonacci(n)
    n번째 피보나치 수를 반환한다.



help 함수를 실행하면 데코레이터로 감싸진 함수에 대해서도 원하는 결과를 볼 수 있다.

기억해야 할 내용
- 파이썬 데코레이터는 실행 시점에 함수가 다른 함수를 변경할 수 있게 해주는 구문이다.
- 데코레이터를 사용하면 디버거 등 인트로스펙션을 사용하는 도구가 잘못 작동할 수 있다.
- 직접 데코레이터를 구현할 떄 인트로스펙션에서 문제가 생기지 않길 바란다면 functools 내장 모듈의 wraps 데코레이터를 사용하라.